In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

try:
    df = pd.read_csv('stackoverflow_200.csv')
except FileNotFoundError:
    print("Error: 'stackoverflow_200.csv' not found.")
    exit()

# Handling Missing Values: Fill missing tags with 'General'
df['top_tags'] = df['top_tags'].fillna('general')

# Handling Outliers: Extreme high reputation users (like Arjun Rossi at 1.6M) 
# can blind the model to other good candidates. We cap these at the 95th percentile.
cols_to_cap = ['reputation', 'total_answer_score_fetched', 'avg_score_per_answer']
for col in cols_to_cap:
    limit = df[col].quantile(0.95)
    df[col] = np.where(df[col] > limit, limit, df[col])

# Architect Metric: Count unique domains to measure "Technical Breadth"
df['tag_breadth'] = df['top_tags'].apply(lambda x: len(x.split(',')))

# Architecture Keyword Search (Infrastructure & System Design)
arch_keywords = ['azure', 'aws', 'kubernetes', 'sql-server', 'database', 'asynchronous', 'git-submodules']
df['arch_keywords_count'] = df['top_tags'].apply(lambda x: sum(1 for kw in arch_keywords if kw in x.lower()))

scaler = MinMaxScaler()
# Normalize all metrics to a 0-1 scale for fair mathematical comparison
metrics = ['reputation', 'accepted_answer_ratio', 'avg_score_per_answer', 'tag_breadth', 'arch_keywords_count']
df_scaled = pd.DataFrame(scaler.fit_transform(df[metrics]), columns=metrics)

# DEVELOPER: Quality/Correctness Relationship
# We prioritize how often their code is "Accepted" as the right solution.
df['Developer_Score'] = (df_scaled['accepted_answer_ratio'] * 0.7) + (df_scaled['avg_score_per_answer'] * 0.3)

# SENIOR DEVELOPER: Authority/Impact Relationship
# We prioritize total Reputation and historical expertise.
df['Senior_Developer_Score'] = (df_scaled['reputation'] * 0.6) + (df_scaled['avg_score_per_answer'] * 0.4)

# SOLUTION ARCHITECT: Breadth/Complexity Relationship
# We prioritize knowledge across many domains and specific infra tags.
df['Solution_Architect_Score'] = (df_scaled['arch_keywords_count'] * 0.5) + (df_scaled['tag_breadth'] * 0.3) + (df_scaled['reputation'] * 0.2)

def get_reason(role, row):
    if role == "Developer":
        return f"High execution reliability. Has a {row['accepted_answer_ratio']:.0%} acceptance rate for their code solutions."
    if role == "Senior Developer":
        limit_rep = int(row['reputation'])
        return f"Established technical authority. Has earned {limit_rep}+ reputation points through peer-validated solutions."
    if role == "Solution Architect":
        return f"Vast technical breadth. Specialized in {row['tag_breadth']} domains including infra-heavy tags: {row['top_tags'][:50]}..."

roles = [("Developer", "Developer_Score"), 
         ("Senior Developer", "Senior_Developer_Score"), 
         ("Solution Architect", "Solution_Architect_Score")]

for role_label, score_col in roles:
    print(f"\n\nTOP 3 {role_label.upper()} CANDIDATES\n")
    top_3 = df.sort_values(score_col, ascending=False).head(3)
    
    for i, (_, row) in enumerate(top_3.iterrows()):
        print(f"{i+1}. {row['display_name']}")
        print(f"   BASIS: {get_reason(role_label, row)}\n")



TOP 3 DEVELOPER CANDIDATES

1. Sanawizard725
   BASIS: High execution reliability. Has a 80% acceptance rate for their code solutions.

2. Mila Taylor
   BASIS: High execution reliability. Has a 80% acceptance rate for their code solutions.

3. Vihaanstack558
   BASIS: High execution reliability. Has a 78% acceptance rate for their code solutions.



TOP 3 SENIOR DEVELOPER CANDIDATES

1. Sana Wright
   BASIS: Established technical authority. Has earned 1109771+ reputation points through peer-validated solutions.

2. Aditi Bianchi
   BASIS: Established technical authority. Has earned 1050656+ reputation points through peer-validated solutions.

3. Zoebyte853
   BASIS: Established technical authority. Has earned 727484+ reputation points through peer-validated solutions.



TOP 3 SOLUTION ARCHITECT CANDIDATES

1. Ethandev997
   BASIS: Vast technical breadth. Specialized in 7 domains including infra-heavy tags: azure:301018(30000), tidyverse:311093(30000), join...

2. Sana Taylor
   BAS